<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 5. Interactive Pipelines</h2></div>

In [ ]:
import numpy as np
import pandas as pd
import panel as pn
import holoviews as hv
from datashader.utils import lnglat_to_meters

pn.extension('tabulator', template='material')

pn.state.template.sidebar_width = 250
pn.config.sizing_mode = 'stretch_width'

import hvplot.pandas # noqa

## Panel widgets

In this notebook, we will want to drive our visualizations with interactive widgets. For this, we will need a widget library where we will be using the [panel](https://panel.holoviz.org/) library.

### Float slider

For instance, let us create a float slider to specify an earthquake magnitude between zero and nine:

In [ ]:
mag_slider = pn.widgets.FloatSlider(name='Magnitude', start=0, end=9, value=6)
mag_slider

We can get the value of this slider from the `.value` parameter:

In [ ]:
mag_slider.value

Try moving the slider around and rerunning the cell above to access the current slider value.

## Date range slider

Panel has many different widgets and you can see a reference gallery of them [here](https://panel.holoviz.org/reference/index.html#widgets). Let's now make a widget to specify a date range:

In [ ]:
date_range = pn.widgets.DateRangeSlider(name='Date', 
                                        start=pd.Timestamp('2000-01-31'), 
                                        end=pd.Timestamp('2018-12-01'))
date_range

Now we can access the value of this slider:

In [ ]:
date_range.value

As this widget is specifying a range, this time the value is specified as a tuple. You can get the components of the tuple directly via the `value_start` and `value_end` parameters respectively:

In [ ]:
f'Start is at {date_range.value_start} and the end is at {date_range.value_end}'

Once again, try specifying different ranges with the widgets and rerunning the cell above.

## The `.interactive` interface

To use the .`interactive` interface from `hvplot`, first we need a `DataFrame` which we load from our earthquake data as before:

In [ ]:
df = pd.read_parquet('../data/earthquakes.parq')
df = df.set_index('time').tz_convert(None).reset_index()
df.head(n=3)

After importing `hvplot.pandas`, we now have an `.interactive` method on our `DataFrame` in addition to the `.hvplot` method which allowing us to create an *interactive* `Dataframe`. Here we create an interactive dataframe, specify a `sizing_mode` to control how our visualizations will look later:

In [ ]:
dfi = df.interactive(sizing_mode='stretch_width')
print(dfi) # TODO: hangs without print

This behaves just like a regular `DataFrame` except now you can pass panel *widgets* as arguments to the pandas methods you are familiar with. Let us make the same magnitude and date range widgets as before, where the only difference now is that the start and end dates can now be constrained by the minumum and maximum date found in the data:

In [ ]:
min_mag = pn.widgets.FloatSlider(name='Magnitude', start=0, end=9, value=6)
date = pn.widgets.DateRangeSlider(name='Date', start=df.time.iloc[0], end=df.time.iloc[-1])

Now we can filter this pandas `DataFrame` in the standard way but instead of only being able to specify literal values, we can now refer to the parameter values of the widgets. Interacting with the slider then updates the expression, rerunning the pipeline of operations (here a simple filter).

This filter is a regular pandas mask expression that filters out earthquakes to find those greater than the minimum magnitude value that occur after the start date and before end date. Note that you can refer to the `value`, `value_start` and `value_end` parameters of the widgets with `param.value`, `param.value_start` and `param.value_end` respectively:

In [ ]:
chosen_columns = ['time', 'mag', 'depth', 'latitude', 'longitude', 'place', 'type']

filtered = dfi[
    (dfi['mag']   > min_mag) &
    (dfi['time'] >= date.param.value_start) &
    (dfi['time'] <= date.param.value_end)
]

filtered[chosen_columns].head()

We now have a view of a pandas `DataFrame` but above the table, we see the two widgets that were declared. When the widgets are interacted with, the `DataFrame` uses the widget values to filter the data, selects the chosen columns and displays the result.

*Note that to see the table update, you want to move the start date of the range slider: otherwise, you may not see the table change as the earthquakes are displayed in date order.*

#### Exercise

To specify the minimum earthquake magnitude, we can just specify the `mag` widget as the `value` parameter of this widget is used by default. To be explicit, you may use `mag.param.value` instead if you wish. Try it!

#### Exercise

For readability, seven columns were chosen before displaying the `DataFrame`. Have a look at `df.columns` and pick a different set of columns for display.

## Plotting with `.interactive`

All pandas methods can be made interactive this way, including the built in `.plot` method that uses matplotlib (note, this isn't `.hvplot`!):

In [ ]:
filtered.plot(y='depth', kind='hist', bins=np.linspace(0, 50, 51))

That said, given that `.hvplot` is available, we can use it to benefit from the interactive features offered by Bokeh as well as the compositionality offered by the HoloViews `+` operator:

In [ ]:
mag_hist = filtered.hvplot(
    y='mag', kind='hist', responsive=True, min_height=200
)

depth_hist = filtered.hvplot(
    y='depth', kind='hist', responsive=True, min_height=200
)

mag_hist + depth_hist

These are the same two histograms we saw earlier, expect now we can filter using the Panel widgets.

## Filtering the earthquakes on a map

To display the earthquakes on a map, we will need to work with a new dataframe that has more columns but fewer rows. As in the previous notebooks, we need to project to get `easting` and `northing` so we can overlay our earthquake points on a map. In addition, we will filter down to one year's worth of earthquakes in 2017 (with magnitude `>4`) so that Bokeh can plot all the earthquakes quickly enough:


In [ ]:
subset_df = df[
            (df['mag']   > 4) &
            (df['time'] >= pd.Timestamp('2017-01-01')) &
            (df['time'] <= pd.Timestamp('2018-01-01'))
]

x, y = lnglat_to_meters(subset_df['longitude'], subset_df['latitude'])
subset_projected = subset_df.join([pd.DataFrame({'easting': x}), pd.DataFrame({'northing': y})])

Now we can make a new interactive `DataFrame` from this new subselection:

In [ ]:
subset_dfi = subset_projected.interactive(sizing_mode='stretch_width')

And now we can declare our widgets and use them to filter the interactive `DataFrame` as before:

In [ ]:
date_subrange = pn.widgets.DateRangeSlider(name='Date', 
                                        start=subset_df.time.iloc[0], 
                                        end=subset_df.time.iloc[-1])
mag_subrange = pn.widgets.FloatSlider(name='Magnitude', start=3, end=9, value=3)

filtered_subrange = subset_dfi[
    (subset_dfi['mag']   > mag_subrange) &
    (subset_dfi['time'] >= date_subrange.param.value_start) &
    (subset_dfi['time'] <= date_subrange.param.value_end)
]

Now we can plot the earthquakes on an ESRI tilesource, including the filtering widgets as follows:

In [ ]:
geo = filtered_subrange.hvplot(
    'easting', 'northing', color='mag', kind='points',
    xaxis=None, yaxis=None, responsive=True, min_height=500, tiles='ESRI'
)

geo

## Terminating methods for `.interactive`

We can create our magnitude and depth histograms one this subset of the data as before:

In [ ]:
mag_subhist = filtered_subrange.hvplot(
    y='mag', kind='hist', responsive=True, min_height=200
)

depth_subhist = filtered_subrange.hvplot(
    y='depth', kind='hist', responsive=True, min_height=200
)

combined = mag_subhist + depth_subhist
combined

Note that this looks like a HoloViews layout with some widgets, but this object is *not* a HoloViews object. Instead it is still an `Interative` object:

In [ ]:
type(combined)

If we need a HoloViews `Layout`,  we can build one from the consistuent objects using the `.holoviews()` terminating method on `Interactive`:

In [ ]:
layout = mag_subhist.holoviews() + depth_subhist.holoviews()
layout

This is now a HoloViews object:

In [ ]:
type(layout)

One reason we might want to access the HoloViews objects is to build linked selections from our interactive `DataFrame`:

In [ ]:
ls = hv.link_selections.instance()
ls(mag_subhist.holoviews()) + ls(depth_subhist.holoviews())

We will see how to retain the sliders together with the linked selections in the [Dashboards notebook](./07_Dashboards.ipynb).

For reference, the terminating methods for an `Interactive` object are:

- `.holoviews()`: Give me a HoloViews objects
- `.panel()`:     Give me a Panel ParamFunction

- `.widgets()`:   Give me a layout of widgets associated with this interactive object
- `.layout()`:    Give me the layout of the widgets and display `pn.Column(obj.widgets(), obj.panel())` where `pn.Column` will be described in the [Dashboards notebook](./07_Dashboards.ipynb).

## Conclusion

The `.interactive` method allows you to build pipelines that appear as interactive visualizations with widgets where the widgets are supplied by Panel.